In [1]:
from __future__ import print_function
from __future__ import division
import numpy as np
import sys
from scipy.stats import uniform
from scipy.stats import multivariate_normal
from scipy.stats import halfcauchy
from scipy.stats import invgamma
from scipy.linalg import sqrtm
import scipy.optimize
import random
import numpy.random as npr
import copy
import time
import scipy.stats as sps
#import mpmath as mp
import scipy.special
import os
import numdifftools as nd
from scipy.special import polygamma, gammaln

In [2]:
np.random.seed(1523)
#precision = 150
#mp.mp.prec += precision
def loglike_ind(y,x,theta):
    
    """this function evaluate the individual log likelihood at individual level
     input is whole data set
    """
    
    npar = np.shape(x)[1]
    tmp = x.dot(theta[:npar])
    if np.any(tmp > 700):
        tmp[np.where(tmp>700)]=700
    
    out = (y.T)*tmp - np.log1p(np.exp(tmp))
    
    if(np.isnan(out).any()==True):
        out[np.where(np.isnan(out))] = np.log(1e-100)
    return out  

In [3]:
def loglike_all(y,x,theta):
    """ 
    loglikelihood of whole data set
    """
    npar = np.shape(x)[1]
    ll_ind = loglike_ind(y,x,theta[:npar])
    out = np.sum(ll_ind)
    return out

In [4]:
def hessianll(x,theta):
    """function that evaluate the second derivative of the likelihood at a value of the data , for 1 data point"""
    
    npar = len(x) # number of real parameter
    out = np.zeros([len(theta),len(theta)])
    out[:npar,:npar] = -np.exp(np.dot(x,theta[:npar]))/(1+np.exp(np.dot(x,theta[:npar])))**2*np.outer(x,x)
    return out

In [5]:
def hessianll2(x,theta):
    """function that evaluate the second derivative of the (individual) likelihood at a value of the data """
    npar = np.int(np.shape(x)[1])
    
    tmp = -1/(np.exp(0.5*np.dot(x,theta[:npar]))+np.exp(-0.5*np.dot(x,theta[:npar])))**2
    out = np.zeros([len(x),len(theta),len(theta)])
    out[:,:npar,:npar] = np.array(map(np.multiply,x[:,:,None]*x[:,None,:],tmp))
    
    return out

In [6]:
def sumHessian(x,theta):
    n = len(x)
    H = 0
    for i in range(0,n):
        H = H + hessianll(x[i],theta)
    return H

In [7]:
def hessianPrior(theta,family,par1,par2):
    """function that evaluate the second derivative of the prior at a value of the data """

    if(family== 'gaussian'):
        out = -np.linalg.inv(par2)
        
    else:
        print ("prior not defined !")
        out = np.nan
    return out

In [8]:
def devll_ind(y,x,theta):
    
    """
    function that evaluate the first derivative of the log likelihood at a value of the data
    for individual
    """
    
    #npar = np.shape(data)[1]-1L
    npar = len(x)
    tmp = np.dot(x,theta[:npar])
    if (tmp < -700):
       tmp=-700
    out = np.zeros(len(theta))
    out[:npar] = y*x - x*1/(1+np.exp(-tmp))
    return out  

In [9]:
def devll(y,x,theta):
    """ for a set of observations
    """
    npar = np.shape(x)[1] #number of 'real' parameters- betas
    tmp = np.dot(x,theta[:npar])
    if (np.any(tmp < -700)==True):
       tmp[np.where(tmp<-700)]=-700
    tmp = 1/(1+np.exp(-tmp)) 
    out = np.zeros([len(y),npar])
    out[:,:npar] = np.multiply(x.T,y-tmp).T   
    #out = y*x - np.multiply(x.T,tmp).T
    return out

In [10]:
def sumDev(y,x,theta):
    n = len(y)
    D = 0
    for i in range(0,n):
        D = D+   devll_ind(y[i],x[i],theta)  
    return D

In [11]:
def dprior(theta,family,par1,par2):
    """
    log prior 
    
    for a gaussian prior, par1 is mean, par2 is covariance matrix
    """
    if family =='gaussian':
        out = -0.5*np.dot((theta-par1),np.linalg.solve(par2,(theta-par1)))
        #out = np.log(max(1e-300,multivariate_normal.pdf(theta,par1,par2)))
    else:
        print ('prior not defined')
        out = np.nan
    return out

In [12]:
def gradPrior(theta,prior,npar,priorPar1 = 0, priorPar2  =0):
    
    if prior =='gaussian':
        gradPrior = - np.linalg.solve(priorPar2,theta-priorPar1)
    else:
        print ('prior not defined')
        gradPrior = np.nan
    return gradPrior

In [13]:
def gradU(y,x,theta,prior='uniform',priorPar1=0,priorPar2=0):
    
    """
    function that evaluate the gradient of minus the log posterior 
    """
    npar = np.shape(x)[1]
    const = 1/(1+np.exp(-np.dot(x,theta[:npar])))
    dev_ind = y.reshape(-1,1)*x - const.reshape(-1,1)*x
    gradll= np.zeros(len(theta))
    gradll[:npar] = -np.sum(dev_ind,axis =0) #minus gradient of log likelihood
    gPrior = gradPrior(theta,prior,npar,priorPar1,priorPar2)
    out = gradll  - gPrior
    return out

In [14]:
def potential(y,x,theta,par1,par2,family):
    """ potential energy"""
    U = -(loglike_all(y,x,theta)+dprior(theta,family,par1,par2))
    return U

In [15]:
def kinetic(p,M):
    # technically just minus the log of multivariate normal density with mean 0 and covariance M, without the normalizing constant
    # may be numpy library is faster but just use the formular first
   
    K = 0.5*p.dot(np.linalg.solve(M,p))
    return K

In [17]:
def hmc(y,x, theta,burnin,samples,eps,trajLength,maxSteps,M,priorArgs,adaptArgs,logFile,saveTempOutput=False):
    """ IMPLEMENTATION OF HMC WITH ADAPTIVE WARMUP
        M to be updated using the output from the previous iterations
        eps to be updated continously during burnin
        
    """
    # eps is stepsize (starting value, to be update if burnin>0)
    # L is number of steps (fix)
    # M is covariance matrix of p (to be update if burnin > 0)
    
    # burnin = 0 implies no update at all
    
    #n = len(data)
    niter = burnin + samples
    npar = len(theta)
    nbetas = np.shape(x)[1]
    theta_keep = np.zeros([niter,npar])
    theta_propose = np.zeros([niter,npar])
    acc_rate = np.zeros(niter)
    L_keep = np.zeros(niter)
    L = int(trajLength[0]/eps)
    timePerIter = np.zeros(niter)
    Hdiff = np.zeros(niter)
    # arguments for prior
    pfamily = priorArgs['family']
    priorPar1 = priorArgs['par1']
    priorPar2 = priorArgs['par2']
    
    meanp = np.zeros(len(theta))
    #-------------------------------#
    updateFreq = adaptArgs['updateFreq']
    alpha = adaptArgs['alpha'] #desired acceptance rate
    gamma =adaptArgs['gamma']
    kappa = adaptArgs['kappa']
    updateM = adaptArgs['updateM']
    t0 = adaptArgs['t0']
    Hbar = 0 #Hbar is a sumstat ~ different between the desired acceptance rate and the mean acceptance rate upto time t
    mu = np.log(10*eps)
    logEps = np.log(eps)
    logEpsBar = 0
    #-----------------------------------#
    #
    if(adaptArgs['adapt']==True and burnin>0):
        eps_keep =np.zeros(niter)
        eps_keep[0] = eps
        diagM = adaptArgs['diagM']
        phaseStartPt = adaptArgs['phaseStartPt']
        if(len(phaseStartPt) != len(trajLength)):
            print('phaseEndPt must have same length as trajLength')
            
        phase = 0
    else:
        eps_keep = eps
    #-------------------------------#
    try:
        for i in range(0,niter):
            progress = i*100/niter
            if np.mod(progress,5)==0:
                print(str(progress)+ "% ",end= "")
            if (np.mod(progress,10)==0 and i>0):
                msg = str(progress) + "% ; nsteps now is: " + str(L) + "; mean acc is: " + str(np.mean(acc_rate[:i]))
                print(msg)
                lf = open(logFile,"a")
                lf.write(msg+ '\n')
                lf.close()
                if(saveTempOutput):
                    part = int(progress*0.1)
                    temp = {'par':theta_keep[:i],'eps':eps_keep,'M':M}
                    np.save('output/temp'+ str(part) + '.npy',temp)
                        
            startT = time.time() 
            
               
            p = np.random.multivariate_normal(meanp,M,1)[0]
            thetacurrent = theta
            Hcurrent = -(loglike_all(y,x,theta[:nbetas])+dprior(theta,pfamily,priorPar1,priorPar2))+ kinetic(p,M)
            
            L_keep[i] = L
            # move p by half a step
            p = p-0.5*eps*gradU(y,x,theta,pfamily,priorPar1,priorPar2)
            for s in range(0,L):
                # move position
                theta = theta+ eps*np.linalg.solve(M,p)
                # move momentum
                if s<(L-1):
                    p = p - eps*gradU(y,x,theta,pfamily,priorPar1,priorPar2)
                else:
                    p = p - 0.5*eps*gradU(y,x,theta,pfamily,priorPar1,priorPar2)
                
            # negate p
            p = -p
            theta_propose[i] = theta
            H = -(loglike_all(y,x,theta[:nbetas])+dprior(theta,pfamily,priorPar1,priorPar2))+ kinetic(p,M)
        
            
            reject = False
            Hdiff[i] = Hcurrent - H
            accrate = np.exp(min([0,(Hcurrent-H)]))#
            reject = (np.random.uniform(0,1,1)> accrate)
            if reject==True:
                theta = thetacurrent
                H = Hcurrent
            stopT = time.time()
            timePerIter[i] = stopT- startT
            
            #------------------------------------------------#
            # adjust eps and M if burnin > 0
            # don't evaluate the time getting new eps for adaptive tunning as it's small
            if(burnin >0):
                t = i+ 1
                if(adaptArgs['adapt']==True):
                    if(updateM and np.mod(i+1,updateFreq)==0 ):
                        startT = time.time()
                        if(diagM ==True):
                            var_theta = np.var(theta_keep[int(i/2):(i+1),:],axis = 0) #take half the iteration as the first iters might not be informative
                            M= np.diag(1/var_theta)
                        else:
                            
                            thetaRef = np.mean(theta_keep[int(0.5*i):i,:],axis =0)
                            
                            Hprior = hessianPrior(thetaRef,pfamily,priorPar1,priorPar2)
                            M = -sumHessian(x, thetaRef) - Hprior
                        stopT = time.time()
                        timePerIter[i] = timePerIter[i] + stopT- startT  
                    
                    if(i< burnin):
                        Hbar = (1-1/(t+t0))*Hbar + 1/(t+t0)*(alpha-accrate)
                        logEps = mu - np.sqrt(t)/gamma*Hbar
                        logEpsBar = t**(-kappa)*logEps + (1-t**(-kappa))*logEpsBar
                                                                                                        
                        if(phase < len(phaseStartPt)):
                            if((i+1)==phaseStartPt[phase]):
                                currentTrajLength = trajLength[phase]
                                phase +=1 #next phase is phase 1
                            # reset
                            #if(phase < len(phaseStartPt)):
                            #    mu = np.log(10*eps)
                            #    logEps = np.log(eps)
                            #    logEpsBar = 0
                            #    Hbar = 0 
                        eps = np.min([adaptArgs['maxEps'],np.exp(logEps),currentTrajLength])
                        eps_keep[t] = eps
                        L = min(maxSteps,int(round(currentTrajLength/eps,0)))   
                        if (L==0):
                            print('number of steps reaches 0!')
                            L +=1     
                        if(t==burnin) :
                            # t == burnin
                            eps = np.min([np.exp(logEpsBar),adaptArgs['maxEps'],currentTrajLength])
                            eps_keep[t:] = eps
                            L = min(maxSteps,int(round(trajLength[-1]/eps,0)))
                            L_fix = max(1,L)
                    
                    else:
                        L = L_fix
                else:
                    if t < burnin :
                        L = int(round(trajLength[0]/eps,0))
                    else:
                        L = int(round(trajLength[1]/eps,0))
                                                                
                
            theta_keep[i]= theta
            acc_rate[i] = accrate
    except Warning,w:
        print (str(w))
    except TypeError,e:
        print('type error' + str(e))
    except ValueError,e:
        print('value error'+ str(e))
    except IndexError,e:
       print('Index error'+ str(e))
    except (KeyboardInterrupt, SystemExit):
        print('Bye')
    lf = open(logFile,"a")
    lf.write('Run completed successfully')
    lf.close()
    
    finalpar = {'theta':theta_keep}
    currentSet = {'theta':theta,'iter':i}
    return {'par':finalpar,'acc': acc_rate,'eps':eps_keep,'nsteps':L_keep,'M':M,'Hbar':Hbar,'runTime':timePerIter,
    'Hdiff': Hdiff,'current':currentSet,'proposal':theta_propose,'args':adaptArgs}

SyntaxError: invalid syntax (<ipython-input-17-ba7015e9135c>, line 165)